<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#Linear_svc" data-toc-modified-id="Linear_svc-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Linear_svc</a></span></li><li><span><a href="#Финальное-обучение" data-toc-modified-id="Финальное-обучение-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Финальное обучение</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from pymystem3 import Mystem
from tqdm import notebook
from tqdm.notebook import tqdm
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import numpy as np
from scipy.sparse import vstack
from sklearn.svm import LinearSVC
RANDOM_STATE =42

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

In [3]:
tqdm.pandas()
m = Mystem()

In [4]:
custom_stopwords = set(nltk_stopwords.words('english'))| {
    'would', 'could', 'should', 'might', 'may', 'also', 'us', 'get', 'go', 'like'
}

In [5]:
def clear_text(text):
    text = text.lower()  
    text = re.sub(r'[^a-zA-Zа-яА-Я\s]', ' ', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

def lemmatize(text):
    cleaned = clear_text(text)
    lemmas = m.lemmatize(cleaned)
    lemm_text = "".join(lemmas).strip()
    words = [word for word in lemm_text.split() if word not in custom_stopwords]
    return " ".join(words)

In [6]:
df['clear_lemma'] = df['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [7]:
df.head()

,text,toxic,clear_lemma
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww matches background colour seemingly stuck ...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero chance remember page


In [8]:
X = df['clear_lemma']
y = df['toxic']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE,test_size=0.25)

In [10]:
X_train.shape

(119469,)

In [11]:
X_test.shape

(39823,)

In [12]:
X_train = X_train.values.astype('U')
X_test = X_test.values.astype('U')

In [13]:
tfidf = TfidfVectorizer(
    stop_words=custom_stopwords,
    ngram_range=(1, 1)
)

In [14]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [15]:
X_test_tfidf = tfidf.transform(X_test) 

In [16]:
k = 5  
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Обучение

### Logistic Regression

In [17]:
model = LogisticRegression(solver="saga", max_iter=500, random_state=RANDOM_STATE)

In [18]:
param_grid_lr = {
    'class_weight': ['balanced']
}

In [19]:
gs_lr = GridSearchCV(
    estimator=model,
    param_grid=param_grid_lr,
    cv=kf,
    scoring='f1',  
    n_jobs=-1      
)

In [20]:
gs_lr.fit(X_train_tfidf, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LogisticRegression(max_iter=500, random_state=42,
                                          solver='saga'),
             n_jobs=-1, param_grid={'class_weight': ['balanced']},
             scoring='f1')

In [21]:
best_score = gs_lr.best_score_

In [22]:
print(f"F1-score на валидации: {best_score:.4f}")

F1-score на валидации: 0.7508


### Linear_svc


In [23]:
model = LinearSVC(
    penalty="l2",          
    loss="squared_hinge",  
    dual=False,       
    max_iter=1000,         
    random_state=RANDOM_STATE
)

In [24]:
param_grid_svc = {
    'class_weight': ['balanced'],  
               
}

In [25]:
gs_svc = GridSearchCV(
    estimator=model,
    param_grid=param_grid_svc,
    cv=kf,                
    scoring='f1',         
    n_jobs=-1,            

)

In [26]:
gs_svc.fit(X_train_tfidf, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=LinearSVC(dual=False, random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced']}, scoring='f1')

In [27]:
best_score = gs_svc.best_score_
print(f"F1-score на валидации: {best_score:.4f}")

F1-score на валидации: 0.7616


### Финальное обучение

Так как на валидации лучше оказалась модель linear_svc, то используем её

In [28]:
best_model_svc = gs_svc.best_estimator_

In [29]:
predictions_svc = best_model_svc.predict(X_test_tfidf)
f1_svc = f1_score(y_test, predictions_svc)

In [30]:
print(f"F1-score на тесте: {f1_svc:.4f}")

F1-score на тесте: 0.7658


## Выводы

1. **Предобработка данных**:
- Загружен датасет toxic_comments.csv с текстами комментариев и метками токсичности
- Проведена очистка текста (удаление спецсимволов, приведение к нижнему регистру)
- Выполнена лемматизация для приведения слов к нормальной форме
- Удалены стоп-слова с использованием кастомного списка

2. **Векторизация текста**:
* Использованы униграммы из-за ограничений оперативной памяти


3. **Обучение моделей**
* Logistic Regression: F1-score на валидации: 0.7508
* LinearSVC:           F1-score на валидации: 0.7616
* Использование балансировки классов (class_weight='balanced') оказалось критически важным для достижения хорошего качества

4. **Финальное обучение:**
- F1-score(LinearSVC) на тестовой выборке: 0.7658 что удовлетворяет условиям задачи(F1 ≥ 0.75)


